In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
from random import uniform

# WebDriver 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Headless 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

def get_keyword(text):
    return text.replace(" ", "%20")

def sort_kind(index):
    if index == 1:
        return 'vcount'
    elif index == 2:
        return 'date'
    else:
        return 'none'

keyword = '강아지 기침'  # 크롤링할 키워드 설정
driver.get('https://kin.naver.com/search/list.nhn?query=' + get_keyword(keyword))
time.sleep(uniform(0.1, 1.0))

page_index = 1
f = '2020.07.11'  # 크롤링 시작 일자
t = '2024.09.10'  # 크롤링 종료 일자
period_txt = "&period=" + f + ".%7C" + t + "."
_sort_kind = sort_kind(2)

page_url = []

# 최대 페이지 수 설정
max_pages = 10

while page_index <= max_pages:
    time.sleep(uniform(0.1, 1.0))
    driver.get('https://kin.naver.com/search/list.nhn?' + "&sort=" + _sort_kind + '&query=' + get_keyword(keyword) + period_txt + "&section=kin" + "&page=" + str(page_index))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    ## 수정된 부분 : ico_pro 클래스를 가진 주소만 가져오도록 수정 ##
    tags = soup.find_all('dt')
    for tag in tags:
        a_tag = tag.find('a', class_="_searchListTitleAnchor")  # a 태그 찾기
        img_tag = tag.find('img', class_='ico_pro')  # 같은 dt 내에서 img 태그 찾기
        if a_tag and img_tag:  # a 태그와 img 태그가 둘 다 존재할 때만
            url = a_tag['href']  # href 추출
            page_url.append(url)


    page_index += 1  # 페이지 증가

data = []

for url in page_url:
    driver.get(url)
    time.sleep(uniform(0.5, 1.0))  # 페이지 로딩 시간을 위한 대기
    try:
        # 제목 가져오기 (div 계층을 사용한 접근)
        title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[1]'))
        ).text
    except:
        title = ""

    # "강아지" 키워드가 제목에 포함되어 있는지 확인
    if keyword in title:
        try:
            # 질문 가져오기 (div 계층을 사용한 접근)
            question_txt = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[3]'))
            ).text
        except:
            question_txt = ""
        
        # 답변 추출
        answers = driver.find_elements(By.CSS_SELECTOR, ".se-main-container .se-component-content")
        answer_text = [answer.text for answer in answers]
        
        if not answer_text:  # 답변이 없는 경우도 처리
            answer_text = [""]

        # 작성일 가져오기 (XPath 사용)
        try:
            date = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[2]/span[3]'))
            ).text
        except:
            date = "날짜 없음"
        
        # 전문의 여부 및 전문의 종류 확인 (전문의 배지 여부)
        try:
            expert_badge = driver.find_element(By.CSS_SELECTOR, '#answer_2 > div.profile_card._profileCardArea > a > div.card_info > div.profile_info > div.badge_area > span.badge.expert_job')
            expert_type = expert_badge.text  # 전문의 배지 텍스트 추출 (예: '전문의', '수의사' 등)
        except:
            expert_type = "일반 사용자"  # 전문의 배지가 없는 경우

        # 모든 답변을 저장하며, 전문의 여부 추가
        for answer in answer_text:
            data.append([title, question_txt, answer, date, expert_type])

# 데이터프레임에 전문의 여부 컬럼 추가
dog_df = pd.DataFrame(data, columns=['제목', '질문', '답변', '날짜', '전문의 여부'])
dog_df

# 필요에 따라 데이터를 파일로 저장
# dog_df.to_excel('naver_kin_crawling_result.xlsx', index=False)

driver.quit()


In [19]:
dog_df

,제목,질문,답변,날짜,전문의 여부
0,질문\n강아지 기침소리 때문에 걱정입니다.,강아지 기침소리가 안 좋아요. \n감기 때문인지 기관지 문제인지 모르겠어요.\n도움...,강아지 노화나 감기 기운으로 인해\n기관지가 붓게 되는 경우가 있어요.\n평소 짖는...,작성일\n2024.09.04,일반 사용자
1,질문\n강아지 기침소리 때문에 걱정입니다.,강아지 기침소리가 안 좋아요. \n감기 때문인지 기관지 문제인지 모르겠어요.\n도움...,강아지 기침소리 문제면 일단 기관지 문제라 생각하고\n동물병원 가서 진단 받아보세요...,작성일\n2024.09.04,일반 사용자
2,질문\n강아지 기침,4개월된 미니푸들인데요 갑자기 기침을해요 많이 하진않고 한번씩......... 토요...,4개월 된 미니 푸들이 갑자기 기침을 한다면 여러 가지 원인이 있을 수 있습니다. ...,작성일\n2024.08.26,일반 사용자
3,질문\n아기 강아지 기침,아기 강아지가 기침을 해요\n애기땐 보통 기침? 을 많이 하나요? 2 3분에 한번씩...,아니요ㅜ 혹시 바이러스성 감염 걸렸는지 꼭 병원 가서 확인해보시기 바랍니다..,작성일\n2024.08.24,일반 사용자
4,질문\n어린강아지 기침 2개월케곅,어제는 약해서 약만먹엿는데 담날지나고 보니 점점심해져요 왜그럴까여\n\n어린강아지 ...,"마지막처럼 켁취 정도만 하면 그냥 재채기 하는거라고 생각하심 되는데,, 앞에 콜록콜...",작성일\n2024.08.21,일반 사용자
5,질문\n강아지 기침,이제 2개월 반 된 아이인데\n어제 밤부터 계속 기침해요\n목에 뭐가 걸려서 그런 ...,안녕하세요.\n지식iN 동물의료상담 수의사 새은평동물의료센터장 버드나무 이상철입니다...,작성일\n2024.08.19,수의사
6,질문\n강아지 기침,이제 2개월 반 된 아이인데\n어제 밤부터 계속 기침해요\n목에 뭐가 걸려서 그런 ...,갑자기 기침하는 건가요? 아니면 놀다가 기침하는 건가용?,작성일\n2024.08.19,수의사
7,질문\n강아지 기침 소리,강아지가 이런 기침 소리를 낸지 1주일 정도 됐습니다. 병원가서 엑스레이 찍었는데 ...,병원에서 엑스레이 등으로 검사하고\n별일이 없다고 했다면\n견주는 우선 마음을 놓으...,작성일\n2024.08.17,일반 사용자
8,질문\n강아지 기침,애기가 갑자기 기침을 10초-20초 정도 엄청 심하게 몸이 붕뜰정도로 기침을 하다가...,안녕하세요.\n지식iN 동물의료상담 수의사 새은평동물의료센터장 버드나무 이상철입니다...,작성일\n2024.08.14,일반 사용자
9,질문\n강아지 기침,그저께부터 강아지가 기침을 해요 밖에나가면 기침을 안하다가\n에어컨 때문에 그런건가...,그런거 같아여,작성일\n2024.08.08,일반 사용자


In [1]:
<dt>
	<a href="https://kin.naver.com/qna/detail.naver?d1id=8&amp;dirId=80511&amp;docId=475814954&amp;qb=6rCV7JWE7KeA&amp;enc=utf8&amp;section=kin.qna_ency&amp;rank=1&amp;search_sort=0&amp;spq=0" target="_blank" class="_nclicks:kin.txt _searchListTitleAnchor"><b>강아지</b> 진드기 옮기나요??</a>
		<img class="ico_pro" height="14" width="53" alt="수의사답변" src="https://ssl.pstatic.net/static/kin/09renewal/ico_proanswer_vet.gif">
</dt>

<dt>
	<a href="https://kin.naver.com/qna/detail.naver?d1id=8&amp;dirId=80511&amp;docId=475406096&amp;qb=6rCV7JWE7KeA&amp;enc=utf8&amp;section=kin.qna_ency&amp;rank=4&amp;search_sort=0&amp;spq=0" target="_blank" class="_nclicks:kin.txt _searchListTitleAnchor"><b>강아지</b> 1~5차 접종</a>
</dt>

SyntaxError: invalid syntax (590300099.py, line 1)